In [1]:
import ibm_db
import ibm_db_dbi
import ibm_db_sa
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime as dt
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

# Kết nối với DB2
dsn = "DRIVER={IBM DB2 ODBC DRIVER};DATABASE=bludb;HOSTNAME=10.1.50.25;PORT=50000;PROTOCOL=TCPIP;UID=tpbdatait1;PWD=Tpbdatait1@123;"
hdbc = ibm_db.connect(dsn, "", "")
hdbi = ibm_db_dbi.Connection(hdbc)

# Display option
# pd.options.display.float_format = '{:.2f}'.format
# np.set_printoptions(suppress= True)

# Calculate Global Abnormal

In [2]:
ewallet = pd.read_csv(wslib.mount.get_data_path('twt_ebank_ewallet.csv'), index_col=0)
ewallet['NGAY_GD'] = pd.to_datetime(ewallet['NGAY_GD'])
ewallet['SO_CIF'] = ewallet['SO_CIF'].astype('str').str.zfill(8)
ewallet.rename(columns= {'SO_CIF':'CIF'}, inplace= True)
ewallet.head()

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,LOAI_GD,NGAY_GD,KENH_GIAO_DICH,LOAI_VI,CIF,STATUS,SO_VI,AMOUNT,MA_GIAO_DICH,TRACENUMBER
0,Vi dien tu,2021-09-14,CONFIRM NAP TIEN TU VI,MOMO,00325849,Thanh cong,972152262.0,88000.0,384334498,16577426443
1,Vi dien tu,2021-09-14,CONFIRM NAP TIEN TU VI,MOMO,03859507,Thanh cong,362248159.0,3343000.0,384346323,16577919174
2,Vi dien tu,2021-09-14,CONFIRM NAP TIEN EBANKX,ESB2,09361631,Thanh cong,NaN,682000.0,384349046,HYD3265583921377824
3,Vi dien tu,2021-09-14,CONFIRM NAP TIEN TU VI,AIRPAY,01904282,Thanh cong,917001979.0,173500.0,384351931,2109140300322529
4,Vi dien tu,2021-09-14,CONFIRM NAP TIEN TU VI,MOMO,03357289,Thanh cong,354657793.0,12000.0,384357826,16578251428


In [4]:
ewallet['NGAY_GD'].max()

Timestamp('2021-12-31 00:00:00')

In [ ]:
q = ewallet[ewallet['NGAY_GD']]

In [3]:
# # Calculate
# df_group = ewallet.groupby(['NGAY_GD','CIF'], as_index= False)['AMOUNT'].agg(['sum', 'max', 'min', 'mean', 'count'])
# df_group.reset_index(inplace= True)

# # Chon thoi diem quan sat   
# range_min = pd.to_datetime('2021-12-01')
# range_max = pd.to_datetime('2021-12-31')
# global_amount = pd.DataFrame()
# global_count = pd.DataFrame()

# # Global amount
# for i in pd.date_range(range_min,range_max):
#     df = df_group[df_group['NGAY_GD'] == i].nlargest(100, columns = 'sum')
#     global_amount = global_amount.append(df)
# global_amount.reset_index(inplace= True)
# global_amount.drop(['index'], axis= 1, inplace= True)
# print(global_amount.head(10))

# # Global count
# for i in pd.date_range(range_min, range_max):
#     df = df_group[df_group['NGAY_GD'] == i].nlargest(100, columns = 'count')
#     global_count = global_count.append(df)
# global_count.reset_index(inplace= True)
# global_count.drop(['index'], axis= 1, inplace= True)
# print(lobal_count.head(10))

In [4]:
# wslib.save_data('global_amount.csv', global_amount.to_csv(index=False).encode())
# wslib.save_data('global_count.csv', global_count.to_csv(index=False).encode())

# PCT

In [7]:
# Global_amount
global_amount = pd.read_csv(wslib.mount.get_data_path('global_amount.csv'))
global_amount['CIF'] = global_amount['CIF'].astype('str').str.zfill(8)
global_amount.rename(columns= {'sum':'Max_current'}, inplace= True)
global_amount.head()

,NGAY_GD,CIF,Max_current,max,min,mean,count
0,2021-12-01,02474414,30103000.00,5000000.00,50000.00,3344777.78,9
1,2021-12-01,04006352,25000000.00,5000000.00,5000000.00,5000000.00,5
2,2021-12-01,03449466,24125800.00,5000000.00,10000.00,2010483.33,12
3,2021-12-01,01994343,20363000.00,5000000.00,363000.00,4072600.00,5
4,2021-12-01,09355519,20200000.00,5000000.00,200000.00,4040000.00,5


In [6]:
# Global_count
global_count = pd.read_csv(wslib.mount.get_data_path('global_count.csv'))
global_count['CIF'] = global_count['CIF'].astype('str').str.zfill(8)
global_count.rename(columns= {'sum':'Max_current'}, inplace= True)
global_count.head()

,NGAY_GD,CIF,Max_current,max,min,mean,count
0,2021-12-01,08786107,555001.00,50000.00,10000.00,27750.05,20
1,2021-12-01,05664478,344256.00,30000.00,10000.00,18118.74,19
2,2021-12-01,03272149,1640000.00,200000.00,50000.00,91111.11,18
3,2021-12-01,03928410,624000.00,230000.00,10000.00,34666.67,18
4,2021-12-01,03934694,13760000.00,2000000.00,60000.00,764444.44,18


In [7]:
print('So CIF Global_abnormal_KLGD:', len(global_amount['CIF'].unique()))
print('So CIF Global_abnormal_SLGD:', len(global_count['CIF'].unique()))

So CIF Global_abnormal_KLGD: 1735
So CIF Global_abnormal_SLGD: 1244


In [8]:
casa = """
SELECT 
	ac_no,
	sum(CASE WHEN txn_drc_f ='C' THEN total_amount ELSE NULL end) AS Amount_IN,
	sum(CASE WHEN txn_drc_f ='D' THEN total_amount ELSE NULL end) AS Amount_OUT,
	sum(CASE WHEN txn_drc_f ='C' THEN number_transaction ELSE NULL end) AS Count_IN,
	sum(CASE WHEN txn_drc_f ='D' THEN number_transaction ELSE NULL end) AS Count_OUT
FROM 
(SELECT 
		ac_no,
		TXN_DRC_F,
		sum(AMT_LCY) AS total_amount,
		count(TRN_REF_NO) AS number_transaction
FROM cso.TTDATA_ST_TXN 
WHERE YEAR(PPN_DT) = '2021' AND MONTH(PPN_DT) = '12'
GROUP BY 
	AC_NO,
	TXN_DRC_F 
) AS sub
GROUP BY ac_no;
"""
casa = pd.read_sql(casa, hdbi)
casa['CIF'] = casa['AC_NO'].str[:8]
casa['Balance'] = casa['COUNT_IN'] - casa['COUNT_OUT']

# Amount
calofmonth = casa.groupby(['CIF'], as_index=False).sum()
calofmonth['Avg_amount_in'] = (calofmonth['AMOUNT_IN'] /31)
calofmonth['Avg_amount_out'] = (calofmonth['AMOUNT_OUT'] /31)
calofmonth['Avg_count_in'] = (calofmonth['COUNT_IN'] /31)
calofmonth['Avg_count_out'] = (calofmonth['COUNT_OUT'] /31)
calofmonth.head()

,CIF,AMOUNT_IN,AMOUNT_OUT,COUNT_IN,COUNT_OUT,Balance,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out
0,00000000,13.00,19800.00,1.00,3.00,-2.00,0.42,638.71,0.03,0.10
1,00000001,125250900.00,124405579.00,4.00,5.00,-1.00,4040351.61,4013083.19,0.13,0.16
2,00000004,1834366779.00,1648855232.00,34.00,69.00,-35.00,59173121.90,53188878.45,1.10,2.23
3,00000005,59.00,19800.00,1.00,3.00,-2.00,1.90,638.71,0.03,0.10
4,00000006,1.00,0.00,1.00,0.00,0.00,0.03,0.00,0.03,0.00


## Global amount

In [9]:
amount_df = global_amount.merge(calofmonth[['CIF', 'Avg_amount_in', 'Avg_amount_out', 'Avg_count_in', 'Avg_count_out', 'Balance']]
                                , left_on='CIF', right_on='CIF', how='left')
amount_df.head()

,NGAY_GD,CIF,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance
0,2021-12-01,02474414,30103000.00,5000000.00,50000.00,3344777.78,9,36789845.94,36885734.71,4.16,8.16,-124.00
1,2021-12-01,04006352,25000000.00,5000000.00,5000000.00,5000000.00,5,4104529.03,5393840.39,0.61,1.35,-23.00
2,2021-12-01,03449466,24125800.00,5000000.00,10000.00,2010483.33,12,598231.68,1298990.03,0.71,2.45,-54.00
3,2021-12-01,01994343,20363000.00,5000000.00,363000.00,4072600.00,5,3263040.52,3276843.26,0.74,1.77,-32.00
4,2021-12-01,09355519,20200000.00,5000000.00,200000.00,4040000.00,5,NaN,NaN,NaN,NaN,NaN


In [10]:
a = amount_df[amount_df['Max_current'] < amount_df['Avg_amount_out']]
print('So CIF co GDTB Casa lon hon Max Current Top-up:', len(a['CIF'].unique()))
print('So CIF khong map duoc voi CASA:', len(amount_df[amount_df['Avg_amount_out'].isnull()]['CIF'].unique()))

So CIF co GDTB Casa lon hon Max Current Top-up: 125
So CIF khong map duoc voi CASA: 243


In [11]:
a['Diff IN'] = a['Max_current'] - a['Avg_amount_in']
a['% amount IN'] = (a['Max_current'] - a['Avg_amount_in'])/a['Max_current']
a['Diff out'] = a['Max_current'] - a['Avg_amount_out']
a['% amount OUT'] = (a['Max_current'] - a['Avg_amount_out'])/a['Max_current']
pct_a = a.quantile(np.linspace(.1,1,9,0))
pct_a

/tmp/1000700000/ipykernel_300/664348040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['Diff IN'] = a['Max_current'] - a['Avg_amount_in']
/tmp/1000700000/ipykernel_300/664348040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['% amount IN'] = (a['Max_current'] - a['Avg_amount_in'])/a['Max_current']
/tmp/1000700000/ipykernel_300/664348040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Diff IN,% amount IN,Diff out,% amount OUT
0.10,15992724.90,3000000.00,112100.00,2450906.25,3.00,18313250.32,18733187.10,0.84,1.90,-235.90,-55501977.19,-2.94,-58451394.52,-2.92
0.20,17086067.20,3000000.00,500000.00,2833333.33,4.00,20362878.10,20362564.52,1.25,2.81,-141.00,-29861716.50,-1.62,-28426031.98,-1.62
0.30,18000000.00,5000000.00,1000000.00,3000000.00,4.00,20890942.06,20982984.34,1.42,3.42,-103.00,-12752324.54,-0.78,-12797633.37,-0.76
0.40,19000000.00,5000000.00,1643600.00,3537240.00,4.00,21785084.00,22145695.26,2.03,4.11,-91.60,-9336381.62,-0.48,-9560256.35,-0.48
0.50,19999999.00,5000000.00,2400000.00,4009791.50,4.00,25764458.48,26705875.71,2.31,4.79,-71.00,-5764458.48,-0.29,-6705875.71,-0.34
0.60,20000000.00,5000000.00,3999077.80,4995622.45,5.00,28488623.97,28904251.61,2.77,5.61,-46.20,-2864942.61,-0.17,-3313133.55,-0.17
0.70,20000000.00,5000000.00,5000000.00,5000000.00,6.00,31047087.96,31224056.78,2.97,6.23,-34.90,-2306460.97,-0.12,-2226603.03,-0.12
0.80,20000000.00,5000000.00,5000000.00,5000000.00,6.40,49802964.77,48181797.00,4.01,7.10,-22.00,-989879.32,-0.05,-1000341.94,-0.05
0.90,20000000.00,10000000.00,5000000.00,6803333.33,8.00,75501977.19,78451394.52,5.19,10.55,-12.30,-668122.52,-0.04,-794840.32,-0.04


In [12]:
# wslib.save_data('pct_a.csv', pct_a.to_csv(index=False).encode())

In [13]:
amount_data = amount_df[amount_df['Max_current'] > amount_df['Avg_amount_out']]
amount_data['Diff IN'] = amount_data['Max_current'] - amount_data['Avg_amount_in']
amount_data['% amount IN'] = (amount_data['Max_current'] - amount_data['Avg_amount_in'])/amount_data['Max_current']
amount_data['Diff out'] = amount_data['Max_current'] - amount_data['Avg_amount_out']
amount_data['% amount OUT'] = (amount_data['Max_current'] - amount_data['Avg_amount_out'])/amount_data['Max_current']
amount_data.head()

/tmp/1000700000/ipykernel_300/2055161061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amount_data['Diff IN'] = amount_data['Max_current'] - amount_data['Avg_amount_in']
/tmp/1000700000/ipykernel_300/2055161061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amount_data['% amount IN'] = (amount_data['Max_current'] - amount_data['Avg_amount_in'])/amount_data['Max_current']
/tmp/1000700000/ipykernel_300/2055161061.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,NGAY_GD,CIF,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Diff IN,% amount IN,Diff out,% amount OUT
1,2021-12-01,04006352,25000000.00,5000000.00,5000000.00,5000000.00,5,4104529.03,5393840.39,0.61,1.35,-23.00,20895470.97,0.84,19606159.61,0.78
2,2021-12-01,03449466,24125800.00,5000000.00,10000.00,2010483.33,12,598231.68,1298990.03,0.71,2.45,-54.00,23527568.32,0.98,22826809.97,0.95
3,2021-12-01,01994343,20363000.00,5000000.00,363000.00,4072600.00,5,3263040.52,3276843.26,0.74,1.77,-32.00,17099959.48,0.84,17086156.74,0.84
5,2021-12-01,03753617,20130000.00,5000000.00,130000.00,4026000.00,5,3200042.06,3186873.87,1.10,3.13,-63.00,16929957.94,0.84,16943126.13,0.84
6,2021-12-01,00260129,20100000.00,5000000.00,100000.00,4020000.00,5,6032710.39,6039807.16,1.74,1.42,10.00,14067289.61,0.70,14060192.84,0.70


In [38]:
print(len(amount_data['CIF'].unique()))

1388


In [14]:
pct_amount = amount_data.quantile(np.linspace(.1,1,9,0))
pct_amount

,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Diff IN,% amount IN,Diff out,% amount OUT
0.10,16300000.00,3000000.00,200000.00,2585904.76,2.00,1489546.31,1490089.34,0.29,0.62,-77.00,4190135.00,0.22,4379585.52,0.23
0.20,18000000.00,3000000.00,700000.00,2857142.86,4.00,2369594.88,2287032.80,0.42,0.97,-57.00,6828752.13,0.36,6874395.65,0.37
0.30,18818714.80,5000000.00,1500000.00,3333333.33,4.00,3447428.42,3433652.02,0.61,1.29,-43.00,8346649.71,0.44,8405898.13,0.44
0.40,19693409.80,5000000.00,2000000.00,4000000.00,4.00,5135882.92,5012701.94,0.74,1.58,-31.00,9879025.85,0.52,9914630.97,0.53
0.50,20000000.00,5000000.00,3000000.00,4750000.00,4.00,7308255.03,7237882.58,0.94,2.03,-25.00,11640786.48,0.62,11685716.13,0.62
0.60,20000000.00,5000000.00,5000000.00,5000000.00,5.00,9223381.74,9128225.81,1.19,2.58,-18.00,13515685.12,0.74,13628748.39,0.74
0.70,20000000.00,5000000.00,5000000.00,5000000.00,5.00,10718621.90,10712929.03,1.48,2.97,-12.00,15318458.90,0.82,15339945.85,0.82
0.80,20000000.00,5000000.00,5000000.00,5000000.00,7.00,12054265.45,12034649.10,2.00,3.52,-8.00,16787959.17,0.87,16828205.16,0.88
0.90,20000000.00,10000000.00,8160000.00,9000000.00,7.00,14463235.26,14381013.61,2.71,4.35,0.00,18100310.66,0.92,18127658.94,0.92


In [15]:
# wslib.save_data('pct_amount.csv', pct_amount.to_csv(index=False).encode())

## Global count

In [16]:
count_df = global_count.merge(calofmonth[['CIF', 'Avg_amount_in', 'Avg_amount_out', 'Avg_count_in', 'Avg_count_out', 'Balance']]
                              , left_on='CIF', right_on='CIF', how='left')
count_df.head()

,NGAY_GD,CIF,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance
0,2021-12-01,08786107,555001.00,50000.00,10000.00,27750.05,20,NaN,NaN,NaN,NaN,NaN
1,2021-12-01,05664478,344256.00,30000.00,10000.00,18118.74,19,NaN,NaN,NaN,NaN,NaN
2,2021-12-01,03272149,1640000.00,200000.00,50000.00,91111.11,18,854516.71,859728.16,1.94,3.68,-54.00
3,2021-12-01,03928410,624000.00,230000.00,10000.00,34666.67,18,4949567.74,4571788.39,1.74,6.13,-136.00
4,2021-12-01,03934694,13760000.00,2000000.00,60000.00,764444.44,18,12093549.52,12107033.35,3.97,11.84,-244.00


In [31]:
b = count_df[count_df['count'] <= count_df['Avg_count_out']]
print('So CIF co SLGD Casa lon hon Max Current Top-up:', len(b['CIF'].unique()))
print('So CIF khong map duoc voi CASA:', len(count_df[count_df['Avg_count_out'].isnull()]['CIF'].unique()))

So CIF co SLGD Casa lon hon Max Current Top-up: 77
So CIF khong map duoc voi CASA: 73


In [32]:
b['Count_diff_out'] = b['count'] - b['Avg_count_out']
b['% count OUT'] = (b['count'] - b['Avg_count_out'])/b['count']
b['Diff IN'] = b['Max_current'] - b['Avg_amount_in']
b['% amount IN'] = (b['Max_current'] - b['Avg_amount_in'])/b['Max_current']
b['Diff out'] = b['Max_current'] - b['Avg_amount_out']
b['% amount OUT'] = (b['Max_current'] - b['Avg_amount_out'])/b['Max_current']
b.head()

/tmp/1000700000/ipykernel_300/981345373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['Count_diff_out'] = b['count'] - b['Avg_count_out']
/tmp/1000700000/ipykernel_300/981345373.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b['% count OUT'] = (b['count'] - b['Avg_count_out'])/b['count']
/tmp/1000700000/ipykernel_300/981345373.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,NGAY_GD,CIF,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Count_diff_out,% count OUT,Diff IN,% amount IN,Diff out,% amount OUT
7,2021-12-01,04204059,6850000.00,2200000.00,50000.00,402941.18,17,9236132.87,9365182.00,9.16,25.45,-505.00,-8.45,-0.50,-2386132.87,-0.35,-2515182.00,-0.37
24,2021-12-01,04257694,1400000.00,400000.00,19000.00,87500.00,16,2499719.16,2499719.16,4.03,18.52,-449.00,-2.52,-0.16,-1099719.16,-0.79,-1099719.16,-0.79
26,2021-12-01,04365987,1208000.00,250000.00,10000.00,75500.00,16,5570016.97,5570000.23,6.65,24.13,-542.00,-8.13,-0.51,-4362016.97,-3.61,-4362000.23,-3.61
42,2021-12-01,03002117,5900000.00,2000000.00,100000.00,453846.15,13,7523691.48,7523772.81,3.06,14.19,-345.00,-1.19,-0.09,-1623691.48,-0.28,-1623772.81,-0.28
48,2021-12-01,02584702,6910000.00,3085000.00,30000.00,575833.33,12,15199292.16,15286098.29,7.19,23.58,-508.00,-11.58,-0.97,-8289292.16,-1.20,-8376098.29,-1.21


In [33]:
ba = b[b['Diff out'] >0]
pct_ba = ba.quantile(np.linspace(.1,1,9,0))
pct_ba

,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Count_diff_out,% count OUT,Diff IN,% amount IN,Diff out,% amount OUT
0.10,1056000.00,277420.00,10000.00,96000.00,9.00,985341.23,992054.84,0.87,10.06,-471.00,-4.03,-0.34,70658.77,0.05,63945.16,0.05
0.20,1397100.00,997100.00,13500.00,116425.00,10.00,1322800.77,1322835.48,1.52,11.00,-449.00,-3.52,-0.32,74299.23,0.07,74264.52,0.06
0.30,3000000.00,1000000.00,15000.00,248487.21,11.00,2499719.16,2499719.16,1.61,11.13,-376.00,-2.94,-0.24,253549.84,0.09,225648.39,0.08
0.40,3042900.00,1000000.00,15199.00,250000.00,11.00,2741616.48,2774351.61,2.19,14.52,-361.00,-2.52,-0.16,258383.52,0.12,249614.35,0.13
0.50,3478821.00,1000000.00,20000.00,276627.27,12.00,2816690.03,2795623.26,2.81,14.52,-361.00,-1.52,-0.12,410656.48,0.16,462706.74,0.20
0.60,3499137.00,1642810.00,50000.00,300000.00,12.00,3068164.52,3016114.26,2.87,14.52,-361.00,-1.06,-0.12,682446.97,0.20,703513.74,0.20
0.70,4000000.00,2100000.00,50000.00,388793.00,13.00,3068164.52,3016114.26,2.87,14.94,-295.00,-1.00,-0.10,1500280.84,0.31,1500280.84,0.30
0.80,5827009.00,2200000.00,50000.00,448231.46,13.00,3068164.52,3016114.26,2.87,16.03,-294.00,-0.52,-0.04,1581211.19,0.33,1924598.29,0.33
0.90,9754000.00,3500000.00,100000.00,812833.33,14.00,8172788.81,7829401.71,4.03,18.52,-256.00,-0.13,-0.01,2758844.48,0.38,2810894.74,0.38


In [34]:
bb = b[b['Diff out'] <0]
pct_bb = bb.quantile(np.linspace(.1,1,9,0))
pct_bb

,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Count_diff_out,% count OUT,Diff IN,% amount IN,Diff out,% amount OUT
0.10,791620.00,100000.00,10000.00,57692.31,10.00,2499719.16,2499719.16,2.55,11.13,-549.00,-9.58,-0.77,-26642241.13,-10.05,-24877784.19,-8.89
0.20,1025240.00,200000.00,20000.00,78055.56,10.00,2828235.03,2789741.94,3.06,12.59,-508.00,-7.58,-0.57,-15796889.45,-5.37,-16146277.35,-5.21
0.30,1350000.00,200000.00,29400.00,106775.14,11.00,4512743.26,4511130.10,3.43,14.16,-449.00,-6.16,-0.44,-8805593.65,-3.51,-8826813.06,-3.48
0.40,1680000.00,300000.00,50000.00,145000.00,11.00,6094111.71,5869128.95,4.03,14.32,-355.60,-4.29,-0.36,-7040903.61,-2.76,-7089175.48,-2.73
0.50,2185000.00,500000.00,50000.00,186000.00,12.00,7523691.48,7825807.03,4.32,16.29,-345.00,-3.24,-0.27,-4555297.39,-1.99,-4507872.24,-1.99
0.60,2800000.00,500000.80,50000.00,231274.40,13.00,10628292.65,10628679.65,5.55,18.52,-333.00,-2.52,-0.18,-3199577.43,-1.46,-3287083.10,-1.45
0.70,3833000.00,1000000.00,81290.00,291301.67,14.00,15199292.16,15286098.29,6.16,19.29,-293.00,-1.51,-0.13,-2211171.27,-1.08,-2137432.80,-1.09
0.80,5285856.80,1000000.00,100000.00,407088.99,16.00,21247810.58,21223870.97,7.19,23.03,-256.00,-0.97,-0.08,-1535214.53,-0.72,-1535009.59,-0.74
0.90,6983500.00,1744625.90,100800.00,564522.73,16.00,28294009.26,26122784.19,8.10,23.58,-225.00,-0.38,-0.03,-987426.75,-0.45,-949562.96,-0.45


In [21]:
# wslib.save_data('pct_bb.csv', pct_bb.to_csv(index=False).encode())
# wslib.save_data('pct_ba.csv', pct_ba.to_csv(index=False).encode())

In [22]:
count_data = count_df[count_df['count'] > count_df['Avg_count_out']]
count_data['Count_diff_out'] = count_data['count'] - count_data['Avg_count_out']
count_data['% count OUT'] = (count_data['count'] - count_data['Avg_count_out'])/count_data['count']
count_data['Diff IN'] = count_data['Max_current'] - count_data['Avg_amount_in']
count_data['% amount IN'] = (count_data['Max_current'] - count_data['Avg_amount_in'])/count_data['Max_current']
count_data['Diff out'] = count_data['Max_current'] - count_data['Avg_amount_out']
count_data['% amount OUT'] = (count_data['Max_current'] - count_data['Avg_amount_out'])/count_data['Max_current']
count_data.head()

/tmp/1000700000/ipykernel_300/3260594370.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  count_data['Count_diff_out'] = count_data['count'] - count_data['Avg_count_out']
/tmp/1000700000/ipykernel_300/3260594370.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  count_data['% count OUT'] = (count_data['count'] - count_data['Avg_count_out'])/count_data['count']
/tmp/1000700000/ipykernel_300/3260594370.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,NGAY_GD,CIF,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Count_diff_out,% count OUT,Diff IN,% amount IN,Diff out,% amount OUT
2,2021-12-01,03272149,1640000.00,200000.00,50000.00,91111.11,18,854516.71,859728.16,1.94,3.68,-54.00,14.32,0.80,785483.29,0.48,780271.84,0.48
3,2021-12-01,03928410,624000.00,230000.00,10000.00,34666.67,18,4949567.74,4571788.39,1.74,6.13,-136.00,11.87,0.66,-4325567.74,-6.93,-3947788.39,-6.33
4,2021-12-01,03934694,13760000.00,2000000.00,60000.00,764444.44,18,12093549.52,12107033.35,3.97,11.84,-244.00,6.16,0.34,1666450.48,0.12,1652966.65,0.12
5,2021-12-01,04801247,12334997.00,2000000.00,200000.00,685277.61,18,10462630.97,10462643.71,5.29,12.84,-234.00,5.16,0.29,1872366.03,0.15,1872353.29,0.15
6,2021-12-01,03059913,1889443.00,300000.00,31000.00,111143.71,17,4173251.16,4172973.32,4.74,10.52,-179.00,6.48,0.38,-2283808.16,-1.21,-2283530.32,-1.21


In [23]:
count_data_a = count_data[count_data['Diff out'] > 0]
pct_count_a = count_data_a.quantile(np.linspace(.1,1,9,0))
pct_count_a

,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Count_diff_out,% count OUT,Diff IN,% amount IN,Diff out,% amount OUT
0.10,800000.00,100000.00,10013.80,60000.00,10.00,265648.75,272605.96,0.41,1.52,-179.00,4.48,0.38,179468.76,0.13,188487.20,0.13
0.20,1178000.00,200000.00,20000.00,85678.57,11.00,458323.55,455249.61,0.68,2.06,-140.00,6.05,0.48,356198.17,0.26,360739.29,0.25
0.30,1500360.00,266260.00,26400.00,113604.55,11.00,602419.45,606782.35,0.90,2.68,-112.00,7.03,0.57,558008.59,0.35,555362.77,0.35
0.40,1951800.00,400000.00,50000.00,146110.77,12.00,854516.71,859842.53,1.16,3.39,-89.00,7.94,0.63,800948.08,0.44,800125.81,0.43
0.50,2510000.00,500000.00,50000.00,185000.00,13.00,1153826.94,1105230.77,1.48,4.03,-73.00,8.77,0.69,1121030.13,0.52,1121548.39,0.52
0.60,3160800.00,600000.00,50000.00,240666.67,14.00,1442700.32,1413009.07,1.77,4.86,-57.00,9.55,0.74,1620747.98,0.60,1615851.38,0.61
0.70,4437200.00,1000000.00,100000.00,324800.00,16.00,1856661.64,1892649.44,2.16,5.84,-44.00,10.48,0.79,2313685.34,0.68,2326012.85,0.68
0.80,6500000.00,1200000.00,100000.00,500000.00,16.00,2570217.18,2553354.84,2.65,6.94,-35.00,11.48,0.84,3776173.21,0.76,3778645.22,0.76
0.90,10279000.00,2000000.00,200000.00,870566.67,16.00,4383296.65,4293606.45,3.43,9.10,-23.00,13.19,0.88,6855108.72,0.86,6896221.37,0.86


In [24]:
count_data_b = count_data[count_data['Diff out'] < 0]
pct_count_b = count_data_b.quantile(np.linspace(.1,1,9,0))
pct_count_b

,Max_current,max,min,mean,count,Avg_amount_in,Avg_amount_out,Avg_count_in,Avg_count_out,Balance,Count_diff_out,% count OUT,Diff IN,% amount IN,Diff out,% amount OUT
0.10,470000.00,70000.00,10000.00,36289.47,10.00,952786.52,929113.16,0.65,3.03,-297.00,1.48,0.11,-7321369.40,-5.24,-7283369.03,-5.12
0.20,668401.60,100000.00,15000.00,50000.00,10.00,1332955.63,1334839.16,1.00,4.35,-255.00,2.26,0.17,-4264633.98,-2.70,-4305784.47,-2.71
0.30,850967.20,131699.40,20000.00,67950.00,11.00,1699884.48,1731263.99,1.45,5.80,-219.00,3.10,0.24,-2501480.41,-1.64,-2579721.44,-1.66
0.40,1094000.00,200000.00,30000.00,84671.73,12.00,2062457.61,2066587.77,1.81,7.03,-192.00,4.01,0.31,-1548940.06,-1.13,-1583827.30,-1.14
0.50,1343500.00,200000.00,50000.00,101653.85,13.00,2763936.77,2789741.94,2.19,7.90,-165.00,4.94,0.38,-1100604.71,-0.79,-1136314.26,-0.80
0.60,1607200.00,300000.00,50000.00,121310.83,14.00,4036730.43,3952798.61,2.65,8.77,-136.00,5.76,0.46,-773752.33,-0.58,-801364.15,-0.59
0.70,2100000.00,400000.00,50000.00,160012.50,16.00,5318454.56,5384207.42,3.03,9.94,-108.00,6.85,0.53,-561477.65,-0.40,-535736.21,-0.41
0.80,2949000.00,500000.00,100000.00,218650.00,16.00,7377042.58,7442797.29,3.55,11.06,-77.00,8.09,0.63,-319965.51,-0.24,-317503.35,-0.25
0.90,4868000.00,1000000.00,100000.00,365600.00,16.00,10714161.55,11970933.97,4.50,13.23,-49.60,9.72,0.75,-163123.52,-0.12,-161325.38,-0.13


In [35]:
print(len(count_data_b['CIF'].unique()))
print(len(count_data_a['CIF'].unique()))
print(len(ba['CIF'].unique()))
print(len(bb['CIF'].unique()))

449
858
9
77


In [36]:
# wslib.save_data('pct_count_a.csv', pct_count_a.to_csv(index=False).encode())
# wslib.save_data('pct_count_b.csv', pct_count_b.to_csv(index=False).encode())

## Save file

In [ ]:
# wslib.save_data('pct_amount.csv', pct_amount.to_csv(index=False).encode())
# wslib.save_data('pct_count.csv', pct_count.to_csv(index=False).encode())